In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch 
import numpy as np
import torch
from torch.optim import Adam
from model_static_SVI import static_SVI_CP, static_SVI_Tucker

import utils
# import data_loader
import time

In [7]:
# data_file = '../processed_data/beijing_15k.npy'
# data_file = '../processed_data/mvlens_10k.npy'
data_file = '../processed_data/dblp_50k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0
R_U = 2


# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims']
data_dict['device'] = torch.device('cpu')
data_dict['v'] = 1
data_dict['v_time'] = 1


data_dict['U'] = [torch.rand(dim,R_U,1).double() for dim in data_dict['ndims']]

data_dict['R_U'] = R_U
data_dict['gamma_size'] = R_U

model = static_SVI_CP(data_dict)
optimizer = torch.optim.Adam(model.para_list, lr=1e-3)
batch_size = 512


for epoch in range(500):
    curr = 0
    while curr < model.N:
        batch_ind = np.random.choice(model.N, batch_size, replace=False)
        optimizer.zero_grad()
        loss = model.nELBO_batch(batch_ind)
        loss.backward(retain_graph=True)
        optimizer.step()
        curr = curr + batch_size

    if epoch % 2 == 0:
            loss_train,loss_test_rmse,loss_test_MAE = model.model_test(data_dict['te_ind'],torch.tensor(data_dict['te_y']))
            print('train loss %.4f, test loss %.4f'%(loss_train, loss_test_rmse))

train loss 0.7427, test loss 0.7448
train loss 0.5662, test loss 0.5730
train loss 0.4302, test loss 0.4443
train loss 0.3390, test loss 0.3586
train loss 0.2802, test loss 0.3034
train loss 0.2434, test loss 0.2689
train loss 0.2206, test loss 0.2472
train loss 0.2067, test loss 0.2339
train loss 0.1978, test loss 0.2251
train loss 0.1923, test loss 0.2194
train loss 0.1885, test loss 0.2158
train loss 0.1860, test loss 0.2130
train loss 0.1840, test loss 0.2118
train loss 0.1826, test loss 0.2111
train loss 0.1814, test loss 0.2110
train loss 0.1804, test loss 0.2102
train loss 0.1796, test loss 0.2099
train loss 0.1787, test loss 0.2101
train loss 0.1781, test loss 0.2107
train loss 0.1774, test loss 0.2102
train loss 0.1767, test loss 0.2100
train loss 0.1763, test loss 0.2099
train loss 0.1758, test loss 0.2101
train loss 0.1755, test loss 0.2105
train loss 0.1748, test loss 0.2108
train loss 0.1744, test loss 0.2112
train loss 0.1740, test loss 0.2111
train loss 0.1737, test loss

In [8]:
# data_file = '../processed_data/beijing_15k.npy'
data_file = '../processed_data/dblp_50k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0
R_U = 2


# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims'] + [len(full_data['time_uni'])]
data_dict['device'] = torch.device('cpu')
data_dict['v'] = 1
data_dict['v_time'] = 1

data_dict['tr_ind'] = np.concatenate([data_dict['tr_ind'],data_dict['tr_T_disct'].reshape(-1,1)],1)
data_dict['te_ind'] = np.concatenate([data_dict['te_ind'],data_dict['te_T_disct'].reshape(-1,1)],1)

data_dict['U'] = [torch.rand(dim,R_U,1).double() for dim in data_dict['ndims']]

data_dict['R_U'] = R_U
data_dict['gamma_size'] = R_U

model = static_SVI_CP(data_dict)
optimizer = torch.optim.Adam(model.para_list, lr=1e-3)
batch_size = 512


for epoch in range(500):
    curr = 0
    while curr < model.N:
        batch_ind = np.random.choice(model.N, batch_size, replace=False)
        optimizer.zero_grad()
        loss = model.nELBO_batch(batch_ind)
        loss.backward(retain_graph=True)
        optimizer.step()
        curr = curr + batch_size

    if epoch % 2 == 0:
            loss_train,loss_test_rmse,loss_test_MAE = model.model_test(data_dict['te_ind'],torch.tensor(data_dict['te_y']))
            print('train loss %.4f, test loss %.4f'%(loss_train, loss_test_rmse))

/home/zenus/fang/diffusion_tensor/code_fang/model_static_SVI.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.U = [torch.tensor(item,requires_grad=True,device=self.device) for item in hyper_para_dict['U']] # list of mode embedding, fixed and known in this setting


train loss 0.8504, test loss 0.8534
train loss 0.6242, test loss 0.6295
train loss 0.4485, test loss 0.4572
train loss 0.3446, test loss 0.3593
train loss 0.2822, test loss 0.3014
train loss 0.2439, test loss 0.2658
train loss 0.2210, test loss 0.2442
train loss 0.2069, test loss 0.2316
train loss 0.1984, test loss 0.2235
train loss 0.1927, test loss 0.2189
train loss 0.1892, test loss 0.2153
train loss 0.1867, test loss 0.2131
train loss 0.1848, test loss 0.2125
train loss 0.1833, test loss 0.2121
train loss 0.1823, test loss 0.2118
train loss 0.1813, test loss 0.2105
train loss 0.1804, test loss 0.2103
train loss 0.1797, test loss 0.2103
train loss 0.1790, test loss 0.2105
train loss 0.1783, test loss 0.2106
train loss 0.1777, test loss 0.2101
train loss 0.1770, test loss 0.2113
train loss 0.1766, test loss 0.2115
train loss 0.1761, test loss 0.2126
train loss 0.1755, test loss 0.2120
train loss 0.1751, test loss 0.2124
train loss 0.1745, test loss 0.2126
train loss 0.1743, test loss

In [22]:
loss_train,loss_test_rmse,loss_test_MAE = model.model_test(data_dict['te_ind'],torch.tensor(data_dict['te_y']))
print('train loss %.4f, test loss %.4f'%(loss_train, loss_test_rmse))

train loss 0.1361, test loss 0.2253


In [2]:
from model_static_CEP import static_CEP_CP



data_file = '../processed_data/beijing_15k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0
R_U = 3


# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims'] + [1461]
data_dict['device'] = torch.device('cpu')
data_dict['v'] = 1
data_dict['v_time'] = 1

data_dict['ind_tr'] = np.concatenate([data_dict['tr_ind'],data_dict['tr_T_disct'].reshape(-1,1)],1)
data_dict['ind_te'] = np.concatenate([data_dict['te_ind'],data_dict['te_T_disct'].reshape(-1,1)],1)

data_dict['y_tr'] = torch.tensor(data_dict['tr_y'])
data_dict['y_te'] = data_dict['te_y']


data_dict['N'] = len(data_dict['tr_ind'])
data_dict['U'] = [torch.rand(dim,R_U,1).double() for dim in data_dict['ndims']]

data_dict['R_U'] = 3
data_dict['gamma_size'] = 3

data_dict['a0'] = 1.0
data_dict['b0'] = 1.0

data_dict['DAMPPING_gamma']=0.1
data_dict['DAMPPING_U']=0.5

model = static_CEP_CP(data_dict)

RuntimeError: Expected 3-dimensional tensor, but got 4-dimensional tensor for argument #1 'batch1' (while checking arguments for bmm)